In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Constuct Dataframes

In [2]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML project/cw/no_overlap/normalized',filename))
    li.append(dataframe)

no_overlap_data = pd.concat(li, axis=0, ignore_index=True)

In [3]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/30_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML project/cw/30_overlap/normalized',filename))
    li.append(dataframe)
thirty_overlap_data = pd.concat(li, axis=0, ignore_index=True)

In [4]:
filenames = os.listdir(r'/Users/apostoloskalatzis/Downloads/ML project/cw/50_overlap/normalized')

li = []
for filename in filenames:
    dataframe=pd.read_csv(os.path.join(r'/Users/apostoloskalatzis/Downloads/ML project/cw/50_overlap/normalized',filename))
    li.append(dataframe)
fifty_overlap_data = pd.concat(li, axis=0, ignore_index=True)

In [5]:
IDS=[4,5,6,7,8,9,11,12,13,16,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35]

In [6]:
ID=[4.0,5.0,6.0,7.0,8.0,9.0,11.0,12.0,13.0,16.0,19.0,20.0,21.0,22.0,
     23.0,24.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0]

# Cognitive Workload Predictions Sliding Window

## Random Forest

In [7]:
def RFLOOCV(data, ids, outcomevar, dropcols, idcolumn):
    """
        Intermediate function. Please use loocvRF function.
            
    """
    #numestimators=300
    #fs=0.00
    #Get important features 
    #listimportances = LOOCV_featureselection(data, ids, outcomevar, dropcols, idcolumn, numestimators)
    #filteredi = listimportances[listimportances['importances'] < fs]
    #filteredi = filteredi['value']
    
    LOOCV_O = str(ids)
    data[idcolumn] = data[idcolumn].apply(str)
    data_filtered = data[data[idcolumn] != LOOCV_O]
    data_cv = data[data[idcolumn] == LOOCV_O]
   
    # Test data - the person left out of training
    data_test = data_cv.drop(columns=dropcols)
    #data_test = data_test.drop(columns=filteredi) #cvf
    X_test = data_test.drop(columns=[outcomevar])
    y_test = data_test[outcomevar] #This is the outcome variable
    
    # Train data - all other people in dataframe
    data_train = data_filtered.drop(columns=dropcols)
    #data_train = data_train.drop(columns=filteredi)
    X_train = data_train.drop(columns=[outcomevar])
    
    feature_list = list(X_train.columns)
    X_train= np.array(X_train)
    y_train = np.array(data_train[outcomevar]) #Outcome variable here
    RF = RandomForestClassifier(n_estimators=50, min_samples_split= 10,
                                 min_samples_leaf=2, max_features='sqrt', max_depth=90, bootstrap=True)
    RF.fit(X_train, y_train)
    predictions = RF.predict(X_test)
    # Use the forest's predict method on the test data
    accuracy = accuracy_score(y_test, predictions)
    pre_score = precision_score(y_test, predictions, pos_label=0, average='binary')
    re_score= recall_score(y_test, predictions, pos_label=0, average='binary')
    f_score = f1_score(y_test, predictions, pos_label=0, average='binary')
    data = {'Accuracy': [accuracy],'Precision': [pre_score], 'Recall':[re_score],
        'F1-Score': [f_score],'Participant' : [ids]}
    results = pd.DataFrame (data, columns = ['Accuracy','Precision','Recall','F1-Score','Participant'])
    return results

### No overlap

In [8]:
li=[]
for i in IDS:
    RF=RFLOOCV(data=no_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')
    li.append(RF)
RFResults_no_overlap = pd.concat(li, axis=0, ignore_index=True) 
RFResults_no_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/RF/no_overlap.csv')

In [9]:
RFResults_no_overlap['F1-Score'].mean()

0.7760073260073261

In [10]:
RFResults_no_overlap['F1-Score'].std()

0.2263592584081449

### 30% Overlap 

In [12]:
li=[]
for i in ID:
    RF=RFLOOCV(data=thirty_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')

    li.append(RF)
RFResults_thirty_overlap = pd.concat(li, axis=0, ignore_index=True) 
RFResults_thirty_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/RF/30_overlap.csv')

In [13]:
RFResults_thirty_overlap['F1-Score'].mean()

0.7772588522588524

In [14]:
RFResults_thirty_overlap['F1-Score'].std()

0.16399413466944776

### 50% Overlap

In [24]:
li=[]
for i in IDS:
    RF=RFLOOCV(data=fifty_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')

    li.append(RF)
RFResults_fifty_overlap = pd.concat(li, axis=0, ignore_index=True) 
RFResults_fifty_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/RF/50_overlap.csv')

In [25]:
RFResults_fifty_overlap['F1-Score'].mean()

0.7545843045843047

In [26]:
RFResults_fifty_overlap['F1-Score'].std()

0.20684821370622528

## Linear SVM

In [27]:
 def LSVMLOOCV(data, ids, outcomevar, dropcols, idcolumn):
    """
        Intermediate function. Please use loocvRF function.
            
    """
    #numestimators=300
    #fs=0.00
    #Get important features 
    #listimportances = LOOCV_featureselection(data, ids, outcomevar, dropcols, idcolumn, numestimators)
    #filteredi = listimportances[listimportances['importances'] < fs]
    #filteredi = filteredi['value']
    
    LOOCV_O = str(ids)
    data[idcolumn] = data[idcolumn].apply(str)
    data_filtered = data[data[idcolumn] != LOOCV_O]
    data_cv = data[data[idcolumn] == LOOCV_O]
   
    # Test data - the person left out of training
    data_test = data_cv.drop(columns=dropcols)
    #data_test = data_test.drop(columns=filteredi) #cvf
    X_test = data_test.drop(columns=[outcomevar])
    y_test = data_test[outcomevar] #This is the outcome variable
    
    # Train data - all other people in dataframe
    data_train = data_filtered.drop(columns=dropcols)
    #data_train = data_train.drop(columns=filteredi)
    X_train = data_train.drop(columns=[outcomevar])
    
    feature_list = list(X_train.columns)
    X_train= np.array(X_train)
    y_train = np.array(data_train[outcomevar]) #Outcome variable here   
    SV = SVC(C=18.3212714912244, class_weight='balanced', gamma=0.00036952926234649913, kernel='linear')
    SV.fit(X_train, y_train)
    predictions = SV.predict(X_test)
    # Use the forest's predict method on the test data
    accuracy = accuracy_score(y_test, predictions)
    pre_score = precision_score(y_test, predictions, pos_label=0, average='binary')
    re_score= recall_score(y_test, predictions, pos_label=0, average='binary')
    f_score = f1_score(y_test, predictions, pos_label=0, average='binary')
    data = {'Accuracy': [accuracy],'Precision': [pre_score], 'Recall':[re_score],
        'F1-Score': [f_score],'Participant' : [ids]}
    results = pd.DataFrame (data, columns = ['Accuracy','Precision','Recall','F1-Score','Participant'])
    return results

### No overlap

In [28]:
li=[]
for i in IDS:
    LSVM = LSVMLOOCV(data=no_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')
    li.append(LSVM)
LSVMResults_no_overlap = pd.concat(li, axis=0, ignore_index=True) 
LSVMResults_no_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/LSVM/no_overlap.csv')

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
LSVMResults_no_overlap['F1-Score'].mean()

0.8149267399267399

In [31]:
LSVMResults_no_overlap['F1-Score'].std()

0.26380570539810744

### 30% Overlap

In [34]:
li=[]
for i in ID:
    LSVM = LSVMLOOCV(data=thirty_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')
    li.append(LSVM)
LSVMResults_thirty_overlap = pd.concat(li, axis=0, ignore_index=True) 
LSVMResults_thirty_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/LSVM/30_overlap.csv')

In [35]:
LSVMResults_thirty_overlap['F1-Score'].mean()

0.749919524919525

In [36]:
LSVMResults_thirty_overlap['F1-Score'].std()

0.2734795699547475

### 50% Overlap

In [37]:
li=[]
for i in IDS:
    LSVM = LSVMLOOCV(data=fifty_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')

    li.append(LSVM)
LSVMResults_fifty_overlap = pd.concat(li, axis=0, ignore_index=True) 
LSVMResults_fifty_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/LSVM/50_overlap.csv')

In [38]:
LSVMResults_fifty_overlap['F1-Score'].mean()

0.7811246445861832

In [39]:
LSVMResults_fifty_overlap['F1-Score'].std()

0.257033731211186

## Non Linear SVM 

In [40]:
 def NLSVMLOOCV(data, ids, outcomevar, dropcols, idcolumn):
    """
        Intermediate function. Please use loocvRF function.
            
    """
    #numestimators=300
    #fs=0.00
    #Get important features 
    #listimportances = LOOCV_featureselection(data, ids, outcomevar, dropcols, idcolumn, numestimators)
    #filteredi = listimportances[listimportances['importances'] < fs]
    #filteredi = filteredi['value']
    
    LOOCV_O = str(ids)
    data[idcolumn] = data[idcolumn].apply(str)
    data_filtered = data[data[idcolumn] != LOOCV_O]
    data_cv = data[data[idcolumn] == LOOCV_O]
   
    # Test data - the person left out of training
    data_test = data_cv.drop(columns=dropcols)
    #data_test = data_test.drop(columns=filteredi) #cvf
    X_test = data_test.drop(columns=[outcomevar])
    y_test = data_test[outcomevar] #This is the outcome variable
    
    # Train data - all other people in dataframe
    data_train = data_filtered.drop(columns=dropcols)
    #data_train = data_train.drop(columns=filteredi)
    X_train = data_train.drop(columns=[outcomevar])
    
    feature_list = list(X_train.columns)
    X_train= np.array(X_train)
    y_train = np.array(data_train[outcomevar]) #Outcome variable here   
    SV = SVC(C=704.3153801257207, class_weight='balanced', gamma=0.0008326787194289269, kernel='rbf')
    SV.fit(X_train, y_train)
    predictions = SV.predict(X_test)
    # Use the forest's predict method on the test data
    accuracy = accuracy_score(y_test, predictions)
    pre_score = precision_score(y_test, predictions, pos_label=0, average='binary')
    re_score= recall_score(y_test, predictions, pos_label=0, average='binary')
    f_score = f1_score(y_test, predictions, pos_label=0, average='binary')
    data = {'Accuracy': [accuracy],'Precision': [pre_score], 'Recall':[re_score],
        'F1-Score': [f_score],'Participant' : [ids]}
    results = pd.DataFrame (data, columns = ['Accuracy','Precision','Recall','F1-Score','Participant'])
    return results

### No overlap

In [41]:
li=[]
for i in IDS:
    NLSVM = NLSVMLOOCV(data=no_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')
    li.append(NLSVM)
NLSVMResults_no_overlap = pd.concat(li, axis=0, ignore_index=True) 
NLSVMResults_no_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/NLSVM/no_overlap.csv')

/Users/apostoloskalatzis/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
NLSVMResults_no_overlap['F1-Score'].mean()

0.7739010989010991

In [44]:
NLSVMResults_no_overlap['F1-Score'].std()

0.25751866785620753

### 30% Overlap

In [46]:
li=[]
for i in ID:
    NLSVM = NLSVMLOOCV(data=thirty_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')
    li.append(NLSVM)
NLSVMResults_thirty_overlap = pd.concat(li, axis=0, ignore_index=True) 
NLSVMResults_thirty_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/NLSVM/30_overlap.csv')

In [47]:
NLSVMResults_thirty_overlap['F1-Score'].mean()

0.7942418692418692

In [48]:
NLSVMResults_thirty_overlap['F1-Score'].std()

0.2435031153796833

### 50% Overlap

In [49]:
li=[]
for i in IDS:
    NLSVM = NLSVMLOOCV(data=fifty_overlap_data, ids=i, outcomevar='label', dropcols=['Unnamed: 0','ID'], idcolumn='ID')

    li.append(NLSVM)
NLSVMResults_fifty_overlap = pd.concat(li, axis=0, ignore_index=True) 
NLSVMResults_fifty_overlap.to_csv('/Users/apostoloskalatzis/Downloads/ML-Project/Results/Cognitive_workload_results/Sliding_method/NLSVM/50_overlap.csv')

In [50]:
NLSVMResults_fifty_overlap['F1-Score'].mean()

0.772136837521453

In [147]:
NLSVMResults_fifty_overlap['F1-Score'].std()

0.21315566965523775